In [1]:
import gmaps
import gmaps.datasets
import ipywidgets as widgets
from IPython.display import display

In [2]:
class Outlet(object):
    def __init__(self, df):
        self._df = df
        self._symbol_layer = None
        
        self._starbucks_layer = self._create_symbols('starbucks','rgba(150,0,0,0.5)')
        self._kfc_layer = self._create_symbols('kfc','rgba(0,150,0,0.2)')
        
        title_widget = widgets.HTML(
            '<h3>KFC and Starbucks locations<h3>'
            '<h4>Data from <a href="http://ratings.food.gov.uk">UK Food Standards Agency</a></h4>'
        )
        
        controls = self._render_controls(True, True)
        map_figure = self._render_map(True, True)
        self._container = widgets.VBox(
            [title_widget, controls, map_figure])
    
    def render(self):
        display(self._container)
    
    def _render_map(self, initial_star, initial_kfc):
        fig = gmaps.figure()
        symbols = self._generate_symbols(True, True)
        self._symbol_layer = gmaps.Markers(markers = symbols)
        fig.add_layer(self._symbol_layer)
        return fig
        
    def _render_controls(self, initial_star, initial_kfc):
        self._star_check = widgets.Checkbox(value=initial_star, description='Starbucks')
        self._kfc_check = widgets.Checkbox(value=initial_kfc, description='KFC')
        
        self._star_check.observe(self._on_controls_change, names='value')
        self._kfc_check.observe(self._on_controls_change, names='value')
        
        controls = widgets.VBox([self._star_check,self._kfc_check])
        return controls
    
    def _on_controls_change(self, obj):
        include_star = self._star_check.value
        include_kfc = self._kfc_check.value
        symbols = self._generate_symbols(include_star, include_kfc)
        self._symbol_layer.markers = symbols
        
    def _generate_symbols(self, include_star, include_kfc):
        symbols = []
        if include_star:
            symbols.extend(self._starbucks_layer)
        if include_kfc:
            symbols.extend(self._kfc_layer)
        
        return symbols
    
    def _create_symbols(self, chain,colour):
        chain_df = self._df[self._df['chain_name']==chain]
        symbols = [
            gmaps.Symbol(
                location=(latitude,longitude),
                stroke_color = colour,
                fill_color = colour,
                scale = 2
            )
            for latitude,longitude in zip(chain_df['latitude'],chain_df['longitude'])
        ]
        return symbols

In [3]:
df = gmaps.datasets.load_dataset_as_df('starbucks_kfc_uk')
Outlet(df).render()